

---

# **Chapter 5: Managing Complexity and State**

## **Opening Context**
As objects grow in complexity—accumulating optional parameters, validation rules, and construction dependencies—constructor bloat becomes unmaintainable. Meanwhile, some objects are expensive to create from scratch, suggesting we copy existing instances rather than rebuild them. Finally, certain resources (database connections, configuration caches) legitimately require single instances, yet global state introduces testing nightmares and hidden coupling. This chapter addresses three distinct problems: telescoping constructors (Builder), expensive instantiation (Prototype), and instance control (Singleton). We will examine why modern industry practice celebrates the Builder, accepts the Prototype, and increasingly rejects the Singleton in favor of Dependency Injection.

---

## **5.1 Builder Pattern: Constructing Complex Objects Step-by-Step**

### **Intent**
*Separate the construction of a complex object from its representation, allowing the same construction process to create different representations.*

### **The Problem: Telescoping Constructors**
Consider constructing an `HttpRequest` object with multiple optional parameters:

```typescript
// ❌ TELESCOPING CONSTRUCTOR ANTI-PATTERN
class HttpRequest {
    constructor(
        public url: string,
        public method?: string,
        public headers?: Record<string, string>,
        public body?: string,
        public timeout?: number,
        public retries?: number,
        public authToken?: string
    ) {}
}

// Usage is unreadable and error-prone
const request = new HttpRequest(
    "https://api.example.com",
    "POST",
    { "Content-Type": "application/json" },
    '{"key": "value"}',
    5000,
    3,
    "Bearer token123"
);
// Which parameter is 5000? Which is 3? The order is arbitrary and fragile.
```

**Problems**:
1.  **Parameter confusion**: Positional arguments become unreadable with many parameters.
2.  **Invalid combinations**: Some parameters require others (e.g., `body` requires `method` to be POST/PUT), but the constructor cannot enforce this.
3.  **Immutability conflict**: To make the object immutable (best practice), we need all data at construction time, but we don't want a constructor with 10 parameters.

### **The Solution: Builder Structure**

The Builder pattern moves construction logic into a separate `Builder` class, using a **fluent interface** (method chaining) to set parameters incrementally, then building the final immutable object.

### **Implementation: Standard Builder**

```typescript
/**
 * PRODUCT
 * The complex object we want to construct.
 * Kept immutable (readonly) for thread safety and predictability.
 */
class HttpRequest {
    // All fields are readonly (immutable after construction)
    readonly url: string;
    readonly method: 'GET' | 'POST' | 'PUT' | 'DELETE';
    readonly headers: Readonly<Record<string, string>>;
    readonly body?: string;
    readonly timeoutMs: number;
    readonly retryCount: number;
    readonly authToken?: string;

    // Private constructor forces use of Builder
    private constructor(builder: HttpRequestBuilder) {
        this.url = builder.url;
        this.method = builder.method;
        this.headers = Object.freeze({ ...builder.headers }); // Defensive copy
        this.body = builder.body;
        this.timeoutMs = builder.timeoutMs;
        this.retryCount = builder.retryCount;
        this.authToken = builder.authToken;
    }

    // Static factory method to get a new Builder instance
    static builder(): HttpRequestBuilder {
        return new HttpRequestBuilder();
    }

    // Convenience method to create Builder from existing request (for variation)
    toBuilder(): HttpRequestBuilder {
        return new HttpRequestBuilder(this);
    }
}

/**
 * BUILDER
 * Responsible for step-by-step construction and validation.
 */
class HttpRequestBuilder {
    // Default values for optional parameters
    url: string = "";
    method: 'GET' | 'POST' | 'PUT' | 'DELETE' = 'GET';
    headers: Record<string, string> = {};
    body?: string;
    timeoutMs: number = 30000; // Default 30s
    retryCount: number = 0;
    authToken?: string;

    // Constructor for fresh builds
    constructor() {}

    // Constructor for rebuilding from existing request (Prototype-like)
    constructor(request: HttpRequest) {
        this.url = request.url;
        this.method = request.method;
        this.headers = { ...request.headers };
        this.body = request.body;
        this.timeoutMs = request.timeoutMs;
        this.retryCount = request.retryCount;
        this.authToken = request.authToken;
    }

    // Fluent setters (return this for chaining)
    setUrl(url: string): this {
        this.url = url;
        return this;
    }

    setMethod(method: 'GET' | 'POST' | 'PUT' | 'DELETE'): this {
        this.method = method;
        return this;
    }

    addHeader(key: string, value: string): this {
        this.headers[key] = value;
        return this;
    }

    setBody(body: string): this {
        this.body = body;
        return this;
    }

    setTimeout(ms: number): this {
        this.timeoutMs = ms;
        return this;
    }

    setRetries(count: number): this {
        this.retryCount = count;
        return this;
    }

    setAuthToken(token: string): this {
        this.authToken = token;
        return this;
    }

    /**
     * BUILD METHOD
     * Finalizes construction, performs validation, returns immutable Product.
     */
    build(): HttpRequest {
        // Validation logic centralized here, not scattered in constructor
        if (!this.url) {
            throw new Error("URL is required");
        }
        if (this.body && !['POST', 'PUT'].includes(this.method)) {
            throw new Error(`Body not allowed for ${this.method} requests`);
        }
        if (this.timeoutMs < 0) {
            throw new RangeError("Timeout must be non-negative");
        }

        return new HttpRequest(this);
    }
}

// Usage: Readable, flexible, type-safe
const request = HttpRequest.builder()
    .setUrl("https://api.example.com/users")
    .setMethod("POST")
    .addHeader("Content-Type", "application/json")
    .setBody('{"name": "John"}')
    .setTimeout(5000)
    .setRetries(3)
    .setAuthToken("Bearer abc123")
    .build();

console.log(request.url); // Accessible, but immutable

// Creating variation of existing request (Preserving immutability)
const putRequest = request.toBuilder()
    .setMethod("PUT")
    .setBody('{"name": "Jane"}')
    .build();
```

**Code Explanation**:

- **Private Constructor**: The `HttpRequest` constructor is private, forcing construction through `HttpRequestBuilder`. This ensures all objects are validated and fully initialized.
- **Fluent Interface**: Methods like `setUrl()` return `this`, enabling method chaining for readable "sentence-like" construction.
- **Validation in Build**: Complex validation rules (e.g., body only with POST/PUT) are enforced in `build()`, not scattered across setters or the product class.
- **Immutability**: The product is immutable (`readonly` fields), making it thread-safe and preventing invalid state changes after construction.
- **Defensive Copying**: The builder passes copies of objects (e.g., `headers`) to prevent external modification after building.

### **Advanced Pattern: The Director**

When constructing objects follows a specific algorithm or recipe, a **Director** class encapsulates the construction steps, separating *how* to build from *what* to build.

```typescript
/**
 * DIRECTOR
 * Encapsulates construction algorithms.
 */
class RequestDirector {
    constructor(private builder: HttpRequestBuilder) {}

    // Specific construction recipe
    constructAuthenticatedPost(url: string, body: string, token: string): HttpRequest {
        return this.builder
            .setUrl(url)
            .setMethod('POST')
            .addHeader('Content-Type', 'application/json')
            .addHeader('Authorization', `Bearer ${token}`)
            .setBody(body)
            .setTimeout(10000)
            .setRetries(2)
            .build();
    }

    // Another recipe
    constructSimpleGet(url: string): HttpRequest {
        return this.builder
            .setUrl(url)
            .setMethod('GET')
            .setTimeout(5000)
            .build();
    }
}

// Usage
const builder = HttpRequest.builder();
const director = new RequestDirector(builder);

const authPost = director.constructAuthenticatedPost(
    "/api/data", 
    '{"key":"val"}', 
    "token123"
);
```

**When to Use Director**:
- Construction involves mandatory steps in a specific order.
- You need multiple "recipes" for constructing similar objects (e.g., `SUVBuilder` vs `SportsCarBuilder` in a car manufacturing system).

### **Industry Note: The Joshua Bloch Builder (Effective Java)**
The pattern shown above is the industry standard popularized by Joshua Bloch. It is superior to:
- **JavaBeans** (`setX` after construction, breaks immutability).
- **Telescoping constructors** (unreadable).
- **Static factory methods** alone (insufficient for many optional parameters).

---

## **5.2 Prototype Pattern: Cloning Objects Efficiently**

### **Intent**
*Specify the kinds of objects to create using a prototypical instance, and create new objects by copying this prototype.*

### **The Problem**
Creating objects from scratch is expensive when:
- Object initialization involves database queries or complex calculations.
- Objects represent game states, documents, or configurations where you need variations of an existing baseline.
- Subclass explosion occurs (e.g., `RedCircle`, `BlueSquare`, `RedSquare`).

### **The Solution: Cloning**
Instead of instantiating via `new`, ask an existing object to clone itself. This requires:
1.  **Shallow Copy**: Copy primitive values and object references (faster, shared nested objects).
2.  **Deep Copy**: Recursively copy all nested objects (slower, fully independent).

### **Implementation: Registry-Based Prototype**

```typescript
/**
 * PROTOTYPE INTERFACE
 * Declares the cloning contract.
 */
interface Cloneable {
    clone(): this; // Returns 'this' type for proper typing
}

/**
 * CONCRETE PROTOTYPE
 * Implements cloning logic.
 */
class Document implements Cloneable {
    constructor(
        public title: string,
        public content: string,
        public metadata: Metadata // Nested object
    ) {}

    /**
     * SHALLOW CLONE
     * Creates new object, but metadata reference is shared!
     */
    shallowClone(): this {
        // Object.assign creates shallow copy
        return Object.assign(Object.create(Object.getPrototypeOf(this)), this);
    }

    /**
     * DEEP CLONE
     * Creates fully independent copy including nested objects.
     */
    clone(): this {
        // Deep copy implementation
        const cloned = Object.assign(Object.create(Object.getPrototypeOf(this)), this);
        cloned.metadata = this.metadata.clone(); // Clone nested object
        return cloned;
    }

    displayInfo(): void {
        console.log(`${this.title}: ${this.content} (Author: ${this.metadata.author})`);
    }
}

class Metadata implements Cloneable {
    constructor(
        public author: string,
        public createdAt: Date,
        public tags: string[]
    ) {}

    clone(): this {
        // Deep copy: new Date, new array
        return Object.assign(Object.create(Object.getPrototypeOf(this)), {
            author: this.author,
            createdAt: new Date(this.createdAt), // New Date object
            tags: [...this.tags] // New array
        });
    }
}

/**
 * PROTOTYPE REGISTRY
 * Maintains cache of prototypical instances.
 * Allows clients to obtain clones without knowing concrete classes.
 */
class DocumentRegistry {
    private prototypes: Map<string, Document> = new Map();

    register(name: string, doc: Document): void {
        this.prototypes.set(name, doc);
    }

    create(name: string): Document {
        const prototype = this.prototypes.get(name);
        if (!prototype) {
            throw new Error(`Unknown prototype: ${name}`);
        }
        return prototype.clone(); // Return deep copy
    }
}

// Usage
const registry = new DocumentRegistry();

// Register base templates (expensive initialization happens once)
const reportTemplate = new Document(
    "Quarterly Report",
    "Executive Summary:\n\n[Data]",
    new Metadata("System", new Date(), ["finance", "quarterly"])
);
registry.register("report", reportTemplate);

const memoTemplate = new Document(
    "Internal Memo",
    "TO: All Staff\nFROM: [Name]\n\n",
    new Metadata("HR", new Date(), ["memo"])
);
registry.register("memo", memoTemplate);

// Client creates documents from prototypes (cheap)
const q1Report = registry.create("report");
q1Report.title = "Q1 2026 Report"; // Modify specific instance
q1Report.metadata.author = "Alice"; // Independent metadata (deep copy)

const q2Report = registry.create("report");
q2Report.title = "Q2 2026 Report";
q2Report.metadata.author = "Bob";

// Verify independence
console.log(q1Report.metadata.author); // "Alice"
console.log(q2Report.metadata.author); // "Bob" (not affected by q1 change)

// Shallow copy demonstration (dangerous)
const shallow = q1Report.shallowClone();
shallow.metadata.author = "Charlie";
console.log(q1Report.metadata.author); // "Charlie" (unexpectedly changed!)
```

**Code Explanation**:

- **`Cloneable` Interface**: Ensures all prototypes implement `clone()`, polymorphism allows registry to work with any document type.
- **Shallow vs Deep**: `shallowClone()` shares `metadata` reference; `clone()` creates independent `Metadata`. The example demonstrates the danger of shallow copying when nested objects are mutable.
- **Registry**: Decouples clients from concrete prototype classes. Client requests "report", registry returns appropriate clone.

### **Modern Alternative: Structured Clone**
In modern JavaScript/TypeScript, `structuredClone()` provides native deep cloning:

```typescript
// Modern approach (JS/TS 2022+)
const deepCopy = structuredClone(original);
```

However, the Prototype pattern remains valuable when:
- Cloning logic needs customization (e.g., resetting certain fields to defaults).
- Using the Registry pattern for template management.
- Working in languages without native deep clone (Java, C++).

---

## **5.3 Singleton Pattern: The Double-Edged Sword of Global State**

### **Intent**
*Ensure a class has only one instance, and provide a global point of access to it.*

### **The Implementation (Traditional)**

```typescript
/**
 * SINGLETON IMPLEMENTATION
 * Strict enforcement of single instance.
 */
class DatabaseConnection {
    // The single instance (static, private)
    private static instance: DatabaseConnection;
    
    // Connection state
    private connected: boolean = false;
    private connectionString: string;

    // Private constructor prevents 'new' operator
    private constructor(connectionString: string) {
        this.connectionString = connectionString;
        this.connect();
    }

    // Global access point
    static getInstance(connectionString?: string): DatabaseConnection {
        if (!DatabaseConnection.instance) {
            if (!connectionString) {
                throw new Error("Connection string required for initial setup");
            }
            DatabaseConnection.instance = new DatabaseConnection(connectionString);
        }
        return DatabaseConnection.instance;
    }

    private connect(): void {
        console.log(`Connecting to ${this.connectionString}...`);
        this.connected = true;
    }

    query(sql: string): any[] {
        if (!this.connected) throw new Error("Not connected");
        console.log(`Executing: ${sql}`);
        return [];
    }

    // Cleanup problematic in Singletons
    disconnect(): void {
        console.log("Disconnecting...");
        this.connected = false;
        // Problem: Other code might still hold reference!
    }
}

// Usage
const db1 = DatabaseConnection.getInstance("postgresql://localhost/db");
const db2 = DatabaseConnection.getInstance(); // Returns same instance

console.log(db1 === db2); // true
```

### **The Problems: Why Singleton is Often an Anti-Pattern**

Despite its popularity, the Singleton pattern creates severe architectural issues recognized by modern industry standards:

#### **1. Violation of Single Responsibility Principle**
The class manages both its primary responsibility (database connection) *and* instance lifecycle management.

#### **2. Hidden Dependencies (Action at a Distance)**
```typescript
class OrderService {
    processOrder(): void {
        // Hidden dependency! No indication OrderService needs DatabaseConnection
        const db = DatabaseConnection.getInstance();
        db.query("INSERT INTO orders...");
    }
}
```
Testing `OrderService` requires the global Singleton to exist, coupling all tests to database state.

#### **3. Testing Nightmares**
```typescript
// ❌ IMPOSSIBLE TO MOCK in unit tests
test("processOrder creates record", () => {
    // Cannot inject mock database!
    // Must use real database or complex global state reset
    const service = new OrderService();
    service.processOrder();
});
```

#### **4. Thread Safety Issues (Multi-threaded Environments)**
In Java/C#, the basic implementation has race conditions during initialization:

```java
// Race condition: Two threads could create two instances
if (instance == null) { // Thread A and B both see null
    instance = new DatabaseConnection(); // Both create instance!
}
```

**Thread-Safe Implementation (Double-Checked Locking)**:

```typescript
class ThreadSafeSingleton {
    private static instance: ThreadSafeSingleton;
    private static lock = new Object();

    static getInstance(): ThreadSafeSingleton {
        if (!ThreadSafeSingleton.instance) {
            // Synchronized block
            synchronized(ThreadSafeSingleton.lock, () => {
                if (!ThreadSafeSingleton.instance) { // Double-check
                    ThreadSafeSingleton.instance = new ThreadSafeSingleton();
                }
            });
        }
        return ThreadSafeSingleton.instance;
    }
}
```
*Note: TypeScript doesn't have synchronized; this shows the conceptual pattern used in Java/C#.*

### **Modern Alternative: Dependency Injection**

The industry-standard replacement for Singleton is **Dependency Injection** combined with **Singleton Scope** (managed by framework):

```typescript
// ❌ OLD: Static Singleton
class BadService {
    db = DatabaseConnection.getInstance();
}

// ✅ MODERN: Injected dependency, singleton lifecycle managed by container
interface Database {
    query(sql: string): any[];
}

class GoodService {
    // Dependency is explicit in constructor
    constructor(private db: Database) {}
    
    processOrder(): void {
        this.db.query("INSERT...");
    }
}

// Framework (e.g., NestJS, Angular, Spring) manages singleton scope
@Injectable({ scope: Scope.DEFAULT }) // Singleton scope
class PostgresDB implements Database {
    // Single instance created and shared by DI container
    query(sql: string): any[] { /* ... */ }
}

// Testability achieved via mocking
test("processOrder creates record", () => {
    const mockDb: Database = {
        query: jest.fn().mockReturnValue([])
    };
    const service = new GoodService(mockDb); // Inject mock
    service.processOrder();
    expect(mockDb.query).toHaveBeenCalled();
});
```

**Benefits of DI over Singleton**:
- **Explicit dependencies**: Constructor shows requirements.
- **Testability**: Easy to inject mocks/fakes.
- **Flexibility**: Can switch between single and multiple instances via configuration, not code change.
- **No global state**: Instance lifecycle managed by container, not static variables.

### **Legitimate Uses of Singleton**
Despite drawbacks, Singleton remains appropriate for:
1.  **True System Resources**: Logging services (though often wrapped), configuration caches, thread pools.
2.  **Hardware Interfaces**: Printer spoolers, file system wrappers where OS enforces single access point.
3.  **Stateless Utilities**: Math libraries, string manipulation (though static methods are often better).

**Rule of Thumb**: If you need Singleton for "convenience" or "global access," use DI instead. If you need it because the hardware/OS enforces singularity, Singleton is justified.

---

## **Chapter Summary**
In this chapter, we examined three patterns handling construction complexity:

1.  **Builder**: Essential for objects with many optional parameters. Enforces immutability, enables readable construction via fluent interfaces, and centralizes validation. The Director variant encapsulates construction algorithms. **Status: Industry standard, highly recommended.**

2.  **Prototype**: Enables object creation via cloning, useful when initialization is expensive or when creating variations of templates. Requires careful distinction between shallow and deep copying. **Status: Specialized use, largely replaced by native cloning in modern languages but valuable for registry patterns.**

3.  **Singleton**: Ensures single instance and global access but violates SRP, creates hidden coupling, and complicates testing. Modern practice replaces it with Dependency Injection using singleton scope. **Status: Considered anti-pattern in application code; acceptable only for system-level resources.**

**Key Decision Framework**:
- Many optional constructor parameters? → **Builder**
- Copy existing complex object? → **Prototype** (or `structuredClone`)
- Need global single instance? → **Dependency Injection with Singleton Scope** (avoid static Singleton)

---

## **Next Chapter Preview**
**Chapter 6: Adapters and Wrappers (Adapter, Facade, Proxy)**

Having completed Creational patterns, we transition to **Structural** patterns—concerned with how classes and objects compose to form larger structures. Chapter 6 introduces the **Adapter** pattern for bridging incompatible interfaces (essential for integrating legacy systems and third-party APIs), the **Facade** pattern for simplifying complex subsystems behind a unified interface, and the **Proxy** pattern for controlling access to objects (enabling lazy loading, access control, and remote communication). These patterns address the inevitable architectural challenge: making independently developed components work together seamlessly.

---

